In [106]:
import numpy as np
import scipy.constants as C

In [107]:
P0=1000 #psi
Pe=14.69 #psi=presion atmosferica
k=1.134514 #1.139544
M=.03819458 # kg/mol
To=1958.5557 #1940.74 #K

In [108]:
#march
def match(Pe,P0,k):
    c=(k-1)/k
    return np.sqrt(2/(k-1)*(1-(Pe/P0)**c))

def RatArea(M1,M2,k):
    a=(k+1)/(2*(k-1))
    b=(k-1)/2
    return M2/M1*(((1+b*M1**2)/(1+b*M2**2))**a)

def Cf(Pe,P0,k):
    a=(k+1)/(k-1)
    return np.sqrt((2*k**2/(k-1)*(2/(k+1))**a)*(1-(Pe/P0)**((k-1)/k)))

def thrust(Cf,At,P0):
    return At*P0*Cf*6894.757*10**(-6)

def Isp(k,Pe,Po,To,M):
    R=C.R
    a=(k-1)/k
    Isp=np.sqrt(2/a*To*(R/M))*(1-(Pe/Po)**a)/C.g
    return Isp

def diametro(A):
    return np.sqrt(4*A/C.pi)

In [109]:
C.R/M

217.68697595313265

In [110]:
m=50 #kg
Mc=0.3
Mt=1
Me=match(Pe,P0,k)
Dc=39.6 #mm
Ac=C.pi*(Dc/2)**2 #mm**2
Dc=diametro(Ac) ## mm

In [111]:
AtAc=RatArea(Mt,Mc,k)
At=Ac*AtAc
AeAt=RatArea(Me,Mt,k)
Ae=At*AeAt
AeAc=RatArea(Me,Mc,k)
Ae2=Ac*AeAc
De2=diametro(Ae2)
De=diametro(Ae)
Dt=diametro(At)
Cf=Cf(Pe,P0,k)
F=thrust(Cf,At,P0)
Isp=Isp(k,Pe,P0,To,M)

In [112]:
print(De2/2,De/2,Dt/2,Dc/2,Mc,Me,Mt,Cf,F) # Dt,Dc=mm match,Cf=sin dimendiones, F=N

25.40526343583541 25.4052634358354 13.707996901102254 19.8 0.3 2.4195001748467524 1 1.6377377465101806 6665.936788408536


In [113]:
print(AeAc,Ac,AeAt,At, Ae) # En mm2

1.6463305026124773 1231.6299839133426 3.434783833016712 590.3340964161292 2027.6700104486495


## Altitud

In [114]:
a=F/m-C.g
t=Isp/F

In [115]:
print(C.g,t, Isp,F)

9.80665 0.01615203739276218 107.66846026416373 6665.936788408536
